In [2]:
import numpy as np
import pymultinest
import batman
from tqdm import tqdm
import os
import time
import threading
import glob

In [3]:
print(os.environ.get('LD_LIBRARY_PATH'))


/global/u2/e/emmayu/Exomoons2025/MultiNest/MultiNest_v3.12_CMake/multinest/lib:/opt/nvidia/hpc_sdk/Linux_x86_64/24.5/cuda/12.4/lib64:/opt/cray/libfabric/1.20.1/lib64


In [2]:
# Load your time, flux, and error arrays from AVG_SAP.dat
data = np.loadtxt('/pscratch/sd/e/emmayu/kipping_exomoons/KIC-3239945/photometry/planet1/AVG_SAP.dat')
time_array = data[:,0]
observed_flux = data[:,1]
flux_err = data[:,2]


In [3]:
# Clean old outputs
output_base = 'your_fit_'
for f in glob.glob(output_base + '*'):
    os.remove(f)

# Estimated max likelihood calls
max_calls = 50000

## Progress bar setup

In [4]:
# Background thread for real-time progress
def monitor_progress(output_base, max_calls):
    log_file = output_base + 'log.txt'
    last_calls = 0
    pbar = tqdm(total=max_calls, ncols=80, position=0, leave=False,
            bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
    while not os.path.exists(output_base + 'post_equal_weights.dat'):
        if os.path.existsa(log_file):
            try:
                with open(log_file) as f:
                    lines = f.readlines()
                # Find most recent line with "Calls:"
                for line in reversed(lines):
                    if "Calls:" in line:
                        calls = int(line.split("Calls:")[1].split()[0])
                        break
                else:
                    calls = last_calls
                if calls > last_calls:
                    pbar.update(calls - last_calls)
                    last_calls = calls
            except Exception:
                pass
        time.sleep(0.5)
    pbar.update(max_calls - last_calls)
    pbar.close()

In [5]:
# Start the monitor thread
progress_thread = threading.Thread(target=monitor_progress, args=(output_base, max_calls))
progress_thread.start()

  0%|                                                                  | 0/50000

In [6]:
def prior(cube, ndim, nparams):
    cube[0] = 5 + cube[0] * 5           # P: 5-10 days (example)
    cube[1] = cube[1] * 1               # tau: 0-1 phase
    cube[2] = cube[2] * 0.2             # p (Rp/Rs)
    cube[3] = cube[3] * 1               # b: 0-1
    cube[4] = 10**(-3 + cube[4]*6)      # rho_star log-uniform 1e-3-1e3
    cube[5] = cube[5] * 1               # q1 limb darkening
    cube[6] = cube[6] * 1               # q2 limb darkening

In [7]:
def generate_mandel_agol_model(P, tau, p, b, rho_star, q1, q2, time_array):
    """
    Parameters:
    - P: orbital period [days]
    - tau: transit time (mid-transit time) [same units as time_array]
    - p: Rp/Rs (planet radius / star radius)
    - b: impact parameter (unitless)
    - rho_star: stellar density [g/cm^3 or solar units, converted below]
    - q1, q2: quadratic limb darkening coefficients in Kipping's parameterization
    - time_array: 1D numpy array of time stamps [days]
    
    Returns:
    - flux: model flux at each time point
    """

    # Constants
    G = 6.67430e-11       # gravitational constant (m^3 kg^-1 s^-2)
    Msun = 1.98847e30     # solar mass (kg)
    Rsun = 6.957e8        # solar radius (m)
    day = 86400.0         # seconds in a day
    
    # Convert stellar density rho_star to SI (kg/m^3) if needed
    # Here assume rho_star is in g/cm^3:
    rho_star_SI = rho_star * 1000  # 1 g/cm3 = 1000 kg/m3
    
    # Calculate semi-major axis in stellar radii (a/Rs) using Kepler's Third Law:
    # a^3 = (G * M_star * P^2) / (4 * pi^2)
    # M_star can be estimated from rho_star and Rs:
    # rho_star = M_star / (4/3 pi Rs^3)
    # => M_star = rho_star * (4/3) * pi * Rs^3
    
    M_star = rho_star_SI * (4/3) * np.pi * Rsun**3  # kg
    a_meters = ((G * M_star * (P * day)**2) / (4 * np.pi**2))**(1/3)
    a_rs = a_meters / Rsun  # semi-major axis in stellar radii
    
    # Calculate inclination from impact parameter: b = (a/Rs) * cos(i)
    incl_rad = np.arccos(b / a_rs)
    incl_deg = np.degrees(incl_rad)
    
    # Convert limb darkening parameters q1, q2 (Kipping 2013) to u1, u2
    u1 = 2 * np.sqrt(q1) * q2
    u2 = np.sqrt(q1) * (1 - 2 * q2)
    
    # Setup batman parameters
    params = batman.TransitParams()
    params.t0 = tau         # time of inferior conjunction (mid-transit)
    params.per = P          # orbital period in days
    params.rp = p           # planet radius (in stellar radii)
    params.a = a_rs         # semi-major axis (in stellar radii)
    params.inc = incl_deg   # orbital inclination in degrees
    params.ecc = 0.0        # eccentricity (assume circular)
    params.w = 90.0         # longitude of periastron (degrees)
    params.u = [u1, u2]     # quadratic limb darkening coefficients
    params.limb_dark = "quadratic"
    
    # Create model
    m = batman.TransitModel(params, time_array)
    flux = m.light_curve(params)
    
    return flux

# Estimate how many iterations you expect
max_calls = 50000  # adjust based on n_live_points, complexity
progress = tqdm(total=max_calls)

def loglike(cube, ndim, nparams):
    progress.update(1)
    P, tau, p, b, rho_star, q1, q2 = cube[:7]
    model_flux = generate_mandel_agol_model(P, tau, p, b, rho_star, q1, q2, time_array)
    chi2 = np.sum(((model_flux - observed_flux)/flux_err)**2)
    return -0.5 * chi2



%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
pymultinest.run(loglike, prior, n_dims=7, n_params=7,
                outputfiles_basename='your_fit_',
                resume=False, verbose=True, n_live_points=50)



581it [05:38, 1518.43it/s]
735it [05:39, 1523.66it/s]
889it [05:39, 1528.19it/s]
043it [05:39, 1529.43it/s]
196it [05:39, 1515.92it/s]
350it [05:39, 1520.15it/s]
503it [05:39, 1519.77it/s]
656it [05:39, 1521.26it/s]
809it [05:39, 1523.66it/s]
962it [05:39, 1519.66it/s]
115it [05:39, 1522.58it/s]
268it [05:40, 1521.43it/s]
421it [05:40, 1515.10it/s]
574it [05:40, 1518.03it/s]
727it [05:40, 1518.39it/s]
879it [05:40, 1513.64it/s]
031it [05:40, 1515.23it/s]
183it [05:40, 1246.39it/s]
334it [05:40, 1313.74it/s]
488it [05:40, 1372.59it/s]
641it [05:40, 1415.55it/s]
793it [05:41, 1443.09it/s]
946it [05:41, 1466.84it/s]
100it [05:41, 1487.30it/s]
251it [05:41, 1462.59it/s]
403it [05:41, 1476.94it/s]
556it [05:41, 1490.05it/s]
708it [05:41, 1496.50it/s]
861it [05:41, 1505.78it/s]
012it [05:41, 1504.37it/s]
163it [05:42, 1504.56it/s]
315it [05:42, 1506.16it/s]
468it [05:42, 1511.04it/s]
621it [05:42, 1515.26it/s]
773it [05:42, 1515.30it/s]
925it [05:42, 1515.60it/s]
078it [05:42, 1518.91it/s]


Acceptance Rate:                        0.001641
Replacements:                                950
Total Samples:                            578971
Nested Sampling ln(Z):             -10891.403369
Importance Nested Sampling ln(Z):     -30.297714



326it [06:57, 638.91it/s]
464it [06:57, 762.21it/s]
600it [06:57, 876.65it/s]
736it [06:57, 980.72it/s]
872it [06:57, 1068.86it/s]
005it [06:57, 1134.30it/s]
137it [06:57, 902.50it/s] 
274it [06:57, 1006.30it/s]
410it [06:58, 1089.99it/s]
546it [06:58, 1158.67it/s]
684it [06:58, 1217.62it/s]
821it [06:58, 1257.92it/s]
956it [06:58, 1282.63it/s]
093it [06:58, 1306.61it/s]
229it [06:58, 1321.93it/s]
367it [06:58, 1337.89it/s]
503it [06:58, 1320.50it/s]
637it [06:58, 1285.86it/s]
767it [06:59, 1253.93it/s]
894it [06:59, 1235.28it/s]
019it [06:59, 1203.17it/s]
140it [06:59, 1188.17it/s]
260it [06:59, 1174.13it/s]
378it [06:59, 1159.83it/s]
496it [06:59, 1164.49it/s]
613it [06:59, 1162.33it/s]
734it [06:59, 1173.82it/s]
852it [06:59, 1162.08it/s]
987it [07:00, 1215.91it/s]
127it [07:00, 1269.92it/s]
267it [07:00, 1307.77it/s]
407it [07:00, 1335.07it/s]
546it [07:00, 1351.11it/s]
686it [07:00, 1362.97it/s]
823it [07:00, 1333.02it/s]
962it [07:00, 1349.48it/s]
103it [07:00, 1365.09it/s]
245i